# Integer Programming: Fantasy Handegg


In this homework, you will use integer programming to select an entry for a fantasy handegg (aka american football) competition. In these competitions, each participant selects players to compose a fantasy team, and the goal is to choose a set that maximizes the number of points these players will score in real-life games.

Each fantasy entry consists of 10 players. The selection of players is subject to two main types of constraints. The first type restricts the choice of players by position. Namely, a feasible entry contains the following number of players per position (you will find this information for each player in the column 'Position'):

*   1 QB player (quarterback)
*   2 FB player (fullback)
*   2 RB player (running back)
*   3 WR player (wide receiver)
*   2 TE player (tight end)


Each player has a fantasy salary (you will find this information for each player in the column 'Salary'), and there is a budget restriction of \$50,000 over the entry. In other words, the sum of the salaries of the players selected in your entry cannot be larger than \$50,000. The goal is to identify a feasible entry that maximizes the expected number of points (you will find this information for each player in the column 'Points'). The extraction and basic filtering of the data have been provided to you (so please don't change that!!!). As output, **you should print the number of fantasy points scored by your entry and a dataframe containing only the entries for the 9 players you selected.**

You will need to solve this exercise in two steps:
- Step 1: Find an optimal solution
- Step 2: Find a second optimal solution by **forbidding the selection of any player that has been selected in the first step**.




**Curiosity**:  Prof. David Bergman, from the OPIM Department, made a lot of money playing this game:
https://today.uconn.edu/2021/01/business-professor-wins-2-5-million-fantasy-football-jackpot-using-concepts-teaches-students/


**Data source (2020: 1-22)**: https://dailyroto.com/nfl-historical-production-fantasy-points-draftkings-fanduel/





## Setup Your Environment/Imports

In [ ]:
# before you do anything...
# mount your drive!
# click folder on the left...
# import modules

%matplotlib inline
from pylab import *

import shutil
import sys
import os.path
import pandas as pd
import numpy as np
from pylab import * # simpler interface to matplotlib

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))

from pyomo.environ import *

## Finding the data we need

In [ ]:
# Step 1: copy & paste the shared link you will get from Google Drive
URL = 'https://drive.google.com/file/d/1RBQuzs6yQ74_4ECagt_-Ws_IeTt2oO9X/view?usp=sharing'

# Step 2: copy & paste the command below (this part of the command is fixed)
fixed_path = 'https://drive.google.com/uc?export=download&id='


# Step 3: Extract the file ID from the URL (this part of the command is fixed)
file_path = URL.split('/')[-2]

# Retrieve the CSV data and build a dataframe
df = pd.read_csv(fixed_path + file_path)

# Basic data cleansing (PLEASE DO NOT CHANGE THIS!!!)
del df['DK Points']
del df['DKSal']
df = df[df["Week"] == 10].reset_index(drop=True)
df.rename(columns = {'P': 'Position', 'FDSal': 'Salary', 'FD Points': 'Points'}, inplace = True)

# First lines of the dataframe
df.head()

,Position,Player,Week,Team,Opp,Opp Rank,Opp Position Rank,Salary,Points
0,QB,Tom Brady,10,TB,CAR,11,7800,6300,34.84
1,RB,Alvin Kamara,10,NO,SF,2,9000,8200,34.80
2,QB,Kyler Murray,10,ARI,BUF,19,8800,8000,30.90
3,QB,Ben Roethlisberger,10,PIT,CIN,17,7400,6200,32.32
4,QB,Josh Allen,10,BUF,ARI,16,8700,7500,29.36


In [ ]:
df.shape

(486, 9)

# Data Extraction

In [ ]:
indices = df.index.values.tolist()

Position = df['Position'].to_list()
Salary = df['Salary'].to_list()
Points = df['Points'].to_list()

In [ ]:
print(Position)

['QB', 'RB', 'QB', 'QB', 'QB', 'RB', 'QB', 'RB', 'RB', 'RB', 'QB', 'WR', 'QB', 'WR', 'WR', 'QB', 'QB', 'WR', 'RB', 'RB', 'WR', 'RB', 'WR', 'QB', 'WR', 'WR', 'WR', 'WR', 'RB', 'QB', 'RB', 'WR', 'WR', 'RB', 'WR', 'WR', 'QB', 'WR', 'QB', 'WR', 'QB', 'WR', 'RB', 'RB', 'QB', 'WR', 'RB', 'QB', 'RB', 'WR', 'WR', 'WR', 'RB', 'WR', 'RB', 'QB', 'RB', 'WR', 'RB', 'WR', 'QB', 'RB', 'TE', 'RB', 'QB', 'WR', 'WR', 'RB', 'WR', 'RB', 'TE', 'QB', 'QB', 'TE', 'QB', 'WR', 'WR', 'RB', 'QB', 'TE', 'RB', 'WR', 'RB', 'WR', 'TE', 'WR', 'WR', 'RB', 'WR', 'QB', 'TE', 'TE', 'TE', 'WR', 'TE', 'TE', 'RB', 'WR', 'WR', 'TE', 'WR', 'WR', 'WR', 'TE', 'QB', 'WR', 'WR', 'RB', 'WR', 'QB', 'WR', 'WR', 'RB', 'RB', 'RB', 'WR', 'WR', 'WR', 'TE', 'WR', 'QB', 'WR', 'RB', 'WR', 'WR', 'RB', 'RB', 'WR', 'RB', 'WR', 'TE', 'QB', 'WR', 'TE', 'TE', 'WR', 'WR', 'RB', 'TE', 'WR', 'TE', 'TE', 'RB', 'WR', 'RB', 'WR', 'WR', 'WR', 'WR', 'TE', 'WR', 'RB', 'WR', 'RB', 'WR', 'WR', 'RB', 'RB', 'TE', 'RB', 'WR', 'WR', 'WR', 'TE', 'WR', 'RB', 'RB

In [ ]:
print(Salary)

[6300, 8200, 8000, 6200, 7500, 6500, 7900, 4600, 5300, 5100, 5800, 4400, 7100, 4700, 7700, 5200, 6600, 4000, 4000, 4000, 5200, 5600, 7500, 6100, 5200, 5800, 3400, 5500, 6800, 6100, 4500, 6000, 6100, 4700, 5100, 5600, 5600, 6300, 5900, 5700, 5200, 4300, 4000, 6700, 5600, 9000, 4900, 6900, 6000, 6800, 3500, 4500, 8900, 3500, 5000, 5700, 6400, 3000, 6600, 6000, 5300, 5100, 4700, 7100, 7700, 3300, 4800, 7700, 7100, 4000, 4100, 5500, 6300, 2500, 6500, 5300, 5800, 5900, 5400, 4600, 4900, 3000, 6300, 5100, 2500, 6500, 3000, 4000, 5600, 5900, 3500, 3300, 4700, 4900, 2800, 2500, 4900, 5000, 6200, 3700, 6900, 3200, 7000, 2500, 5400, 3000, 6400, 4000, 3800, 6400, 5600, 4200, 5400, 4000, 4000, 3800, 4600, 3000, 3000, 6700, 4400, 6600, 4000, 3000, 4500, 6900, 4000, 4000, 5000, 3600, 2500, 6000, 4800, 4200, 5900, 3000, 3000, 5600, 4300, 5400, 4400, 3600, 6600, 5700, 5200, 4500, 4400, 5900, 3300, 2500, 5000, 5100, 6100, 5500, 4200, 3000, 4000, 5000, 3500, 4200, 7600, 3700, 4000, 3100, 7400, 4000, 400

# Solving the problem

## **1. We define the Pyomo Model and Decision Variables:**

In [ ]:
# Declare a Pyomo ConcreteModel.
# Define binary decision variables for each player to indicate whether they are selected in the fantasy team.
# indices = df.index.values.tolist()
# Position = df['Position'].to_list()
# Salary = df['Salary'].to_list()
# Points = df['Points'].to_list()

In [ ]:
# Define the players' indices as a set
players = df.index.values.tolist()

# Define the positions as a set
positions = set(df['Position'])

# Parameters
salary_cap = 50000  # Budget constraint
position_constraints = {'QB': 1, 'FB': 2, 'RB': 2, 'WR': 3, 'TE': 2}  # Position constraints


In [ ]:
# Create a concrete model
model = ConcreteModel()

# Decision variables
model.selected = Var(players, domain=Binary)


In [ ]:
# Objective function
model.objective = Objective(expr=sum(Points[player] * model.selected[player] for player in players), sense=maximize)


<ipython-input-106-e4d162ae9e5e>:2: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  model.objective = Objective(expr=sum(Points[player] * model.selected[player] for player in players), sense=maximize)


In [ ]:
# Position constraints
model.position_constraints = ConstraintList()
for position in positions:
    model.position_constraints.add(
        sum(model.selected[player] for player in players if Position[player] == position) == position_constraints[position]
    )


<ipython-input-107-78fcfe60a47c>:5: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sum(model.selected[player] for player in players if Position[player] == position) == position_constraints[position]


In [ ]:
# Budget constraint
model.budget_constraint = Constraint(expr=sum(Salary[player] * model.selected[player] for player in players) <= salary_cap)


<ipython-input-108-252e6e20216d>:2: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  model.budget_constraint = Constraint(expr=sum(Salary[player] * model.selected[player] for player in players) <= salary_cap)


In [ ]:
# Solve the model
solver = SolverFactory('cbc')
solver.solve(model)

# Extract selected players
selected_players = [player for player in players if value(model.selected[player]) == 1]


In [ ]:
# Print the optimal solution
print("Optimal Solution:")
print("Total Fantasy Points:", value(model.objective))

# Create a DataFrame with selected players
selected_df = df.loc[selected_players]

# Print the DataFrame with selected players
print("\nSelected Players:")
print(selected_df)

Optimal Solution:
Total Fantasy Points: 218.94

Selected Players:
    Position                    Player  Week Team  Opp  Opp Rank  \
0         QB                 Tom Brady    10   TB  CAR        11   
1         RB              Alvin Kamara    10   NO   SF         2   
5         RB               Josh Jacobs    10   LV  DEN         8   
11        WR  Marquez Valdes-Scantling    10   GB  JAX        23   
13        WR              Cole Beasley    10  BUF  ARI        15   
14        WR           DeAndre Hopkins    10  ARI  BUF         9   
62        TE            Rob Gronkowski    10   TB  CAR        13   
73        TE             Cameron Brate    10   TB  CAR        13   
174       FB             Jakob Johnson    10   NE  BAL         1   
460       FB              Danny Vitale    10   NE  BAL         1   

     Opp Position Rank  Salary  Points  
0                 7800    6300   34.84  
1                 9000    8200   34.80  
5                 7500    6500   32.60  
11                550

# Expand the model to compute the second entry

In [ ]:
# Create a list of selected players from the first step
forbidden_players = [0, 1, 5, 11, 13, 14, 62, 73, 174, 460]  # Update with the indices of selected players

# Add constraints to forbid the selection of players chosen in the first step
model.forbidden_constraints = ConstraintList()
for player in forbidden_players:
    model.forbidden_constraints.add(model.selected[player] == 0)


In [ ]:
# Solve the model for the second step
solver.solve(model)

# Print the optimal solution for the second step
print("\nSecond Optimal Solution:")
print("Total Fantasy Points:", value(model.objective))

# Create a DataFrame with selected players for the second step
selected_players_second_step = [player for player in players if value(model.selected[player]) == 1]
selected_df_second_step = df.loc[selected_players_second_step]

# Print the DataFrame with selected players for the second step
print("\nSelected Players for the Second Step:")
print(selected_df_second_step)



Second Optimal Solution:
Total Fantasy Points: 197.02

Selected Players for the Second Step:
    Position              Player  Week Team  Opp  Opp Rank  Opp Position Rank  \
3         QB  Ben Roethlisberger    10  PIT  CIN        17               7400   
7         RB        Nyheim Hines    10  IND  TEN        24               5500   
8         RB     Ronald Jones II    10   TB  CAR        28               6400   
20        WR     Diontae Johnson    10  PIT  CIN        17               5700   
27        WR         Tee Higgins    10  CIN  PIT        25               6100   
35        WR    Justin Jefferson    10  MIN  CHI         2               6600   
70        TE        Hunter Henry    10  LAC  MIA         7               5500   
79        TE        Mark Andrews    10  BAL   NE         2               6400   
224       FB         Gabe Nabers    10  LAC  MIA        20               4700   
228       FB            C.J. Ham    10  MIN  CHI        10               4500   

     Salary  P